In [2]:

import math
import numpy as np

def _geom_analytic_EG(S0, r, q, sigma, ts, observed=None):
    """Analytic E[G] for geometric-average level under GBM.
    observed: dict {t -> S_obs} of already-fixed points (year fractions).
    """
    observed = observed or {}
    N = len(ts)
    # c = (1/N) * sum_{obs} ln S_obs
    c = sum(np.log(observed[t]) for t in ts if t in observed) / N if observed else 0.0
    # future times
    tf = [t for t in ts if t not in observed]
    if not tf:
        return math.exp(c)
    mu_sum = sum(math.log(S0) + (r - q - 0.5*sigma*sigma)*t for t in tf)
    # Var(sum ln S_ti) = sigma^2 * sum_{i,j} min(ti,tj)
    var_sum = 0.0
    for i, ti in enumerate(tf):
        for j, tj in enumerate(tf):
            var_sum += min(ti, tj)
    v = (sigma*sigma) * var_sum / (N*N)
    mu = c + mu_sum / N
    return math.exp(mu + 0.5*v)

def asian_price_mc(S0, K, r, q, sigma, T, N=12, call_put="call", strike_type="fixed",
                   n_paths=200_000, seed=12345, antithetic=True,
                   observed_fixings=None, use_geometric_cv=True):
    """Simple arithmetic Asian pricer via Monte Carlo (GBM).
    - Equally spaced **discrete** fixings from 0 to T (inclusive).
    - Supports fixed-strike and floating-strike payoffs.
    - Optional seasoned handling via observed_fixings: dict {index -> S_obs} OR {t -> S_obs} if float t in year fractions.
    Returns: dict(price, stderr, ci95)
    """
    rng = np.random.default_rng(seed)
    call = call_put.lower() == "call"
    fixed = strike_type.lower() == "fixed"
    observed_fixings = observed_fixings or {}

    # Build fixing times (year fractions) equally spaced over (0, T], N points
    ts = np.linspace(T/N, T, N)

    # Normalize observed_fixings to index->value for convenience
    obs_idx = {}
    if len(observed_fixings) > 0:
        # support both index-based and time-based keys
        for k, v in observed_fixings.items():
            if isinstance(k, int):
                obs_idx[k] = float(v)
            else:
                # time-based: map to nearest index
                idx = int(round((float(k)/T) * N)) - 1
                idx = max(0, min(N-1, idx))
                obs_idx[idx] = float(v)

    # Determine which indices are still stochastic
    to_sim = [i for i in range(N) if i not in obs_idx]

    # Precompute drifts/vols per fixing time
    drift = (r - q - 0.5*sigma*sigma) * ts
    vol = sigma * np.sqrt(ts)

    # Generate normals only for to_sim columns
    Z = rng.standard_normal(size=(n_paths, len(to_sim))) if to_sim else np.zeros((n_paths, 0))
    # Build full Z matrix
    Z_full = np.zeros((n_paths, N))
    for k, i in enumerate(to_sim):
        Z_full[:, i] = Z[:, k]

    # Exact marginals for GBM at fixing times
    S = S0 * np.exp(drift + vol * Z_full)
    if antithetic:
        S_anti = S0 * np.exp(drift + vol * (-Z_full))
        S = np.vstack([S, S_anti])

    # Inject observed fixings (seasoned)
    if obs_idx:
        for i, s in obs_idx.items():
            S[:, i] = s

    # Arithmetic average
    A = S.mean(axis=1)
    S_T = S[:, -1]

    # Payoff
    if fixed:
        payoff = np.maximum(A - K, 0.0) if call else np.maximum(K - A, 0.0)
    else:
        payoff = np.maximum(S_T - A, 0.0) if call else np.maximum(A - S_T, 0.0)

    # Discount
    disc = math.exp(-r * T)
    X = disc * payoff

    # Optional geometric-average control variate
    rho = None
    if use_geometric_cv:
        EG = _geom_analytic_EG(S0, r, q, sigma, ts, observed={ts[i]: s for i, s in obs_idx.items()})
        with np.errstate(divide="ignore"):
            G = np.exp(np.log(np.clip(S, 1e-300, None)).mean(axis=1))
        Y = G
        yv = Y.var(ddof=1)
        if yv > 0:
            covXY = np.cov(X, Y, ddof=1)[0, 1]
            b = covXY / yv
            X = X - b * (Y - EG)
            rho = covXY / (X.std(ddof=1) * Y.std(ddof=1) + 1e-16)

    price = X.mean()
    stderr = X.std(ddof=1) / math.sqrt(len(X))
    ci95 = (price - 1.96*stderr, price + 1.96*stderr)

    return {
        "price": float(price),
        "stderr": float(stderr),
        "ci95": [float(ci95[0]), float(ci95[1])],
        "diagnostics": {"antithetic": bool(antithetic), "geometric_cv": bool(use_geometric_cv), "cv_rho": None if rho is None else float(rho)}
    }

if __name__ == "__main__":
    # Example 1: Unseasoned, fixed-strike call
    resA = asian_price_mc(S0=100, K=100, r=0.03, q=0.01, sigma=0.22, T=1.0, N=12,
                          call_put="call", strike_type="fixed", n_paths=200_000, seed=123, antithetic=True)
    print("Case A:", resA)

    # Example 2: Seasoned, floating-strike put (6 of 8 fixings observed)
    observed = {0:97.20, 1:96.10, 2:94.80, 3:95.40, 4:93.90, 5:92.70}  # first 6 fixings
    resB = asian_price_mc(S0=95, K=None, r=0.0225, q=0.005, sigma=0.28, T=8/52, N=8,
                          call_put="put", strike_type="floating", n_paths=200_000, seed=321,
                          observed_fixings=observed, antithetic=True)
    print("Case B:", resB)


Case A: {'price': 2.8866785441902074, 'stderr': 0.0022445759645915706, 'ci95': [2.8822791752996078, 2.891077913080807], 'diagnostics': {'antithetic': True, 'geometric_cv': True, 'cv_rho': 1.9729877047358217}}
Case B: {'price': 3.54883785299437, 'stderr': 0.006641031057608241, 'ci95': [3.535821432121458, 3.561854273867282], 'diagnostics': {'antithetic': True, 'geometric_cv': True, 'cv_rho': -0.6414567489550341}}


In [4]:
# from asian_simple import asian_price_mc

res = asian_price_mc(
    S0=100, K=100, r=0.03, q=0.01, sigma=0.22,
    T=1.0, N=12, call_put="call", strike_type="fixed",
    n_paths=200_000, seed=123, antithetic=True,
    observed_fixings=None, use_geometric_cv=True
)
print(res["price"], res["stderr"], res["ci95"])


2.8866785441902074 0.0022445759645915706 [2.8822791752996078, 2.891077913080807]
